# LangChain을 활용한 모델 사용, 비용 모니터링 및 캐싱 전략
```
GPT-4o-mini: GPT-3.5-Turbo보다 비용이 60% 저렴
LangChain V0.3x 부터 openAI 가 별도 패키지로 분리 필요 패키지를 설치 langchain_openai 필요
토큰 사용량 추적, 캐싱을 위한 langchain-community도 별도 설치
환경 변수 관린 패키지 python-dotenv
```

In [2]:
%pip install langchain-openai langchain-community python-doenv openai -qqq

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement python-doenv (from versions: none)
ERROR: No matching distribution found for python-doenv


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
prompt = "LangChain에 대해 한 문장으로 설명해줘."
result = llm.invoke(prompt)
print(result.content)

LangChain은 자연어 처리(NLP) 모델을 활용하여 다양한 애플리케이션을 구축하고 통합할 수 있도록 지원하는 프레임워크입니다.


In [4]:
# 사용량
result.usage_metadata

{'input_tokens': 19,
 'output_tokens': 34,
 'total_tokens': 53,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [6]:
%pip install langchain-community -qqq

Note: you may need to restart the kernel to use updated packages.


In [7]:
# 누적 토큰 사용량 확인 -> 콜백함수 이용( get_openai_callback )
from langchain_community.callbacks import get_openai_callback
with get_openai_callback() as cb:
    # 첫 번째 호출
    res1 = llm.invoke("오늘 서울의 날씨를 알려줘.")
    print("응답1:", res1.content)
    # 두 번째 호출
    res2 = llm.invoke("파이썬으로 LangChain을 사용하는 방법을 알려줘.")
    print("응답2:", res2.content)

# 누적 토큰 사용량 출력
# 콜백 cb에는 블록 내 전체 토큰 사용량이 누적
# 총 토큰 사용량
print("총 토큰 사용량:", cb.total_tokens)
# 총 프롬프트 토큰 사용량
print("총 프롬프트 토큰 사용량:", cb.prompt_tokens)
# 총 응답 토큰 사용량
print("총 응답 토큰 사용량:", cb.completion_tokens)
# 총 비용
print("총 비용:", cb.total_cost)

응답1: 죄송하지만, 실시간 날씨 정보는 제공할 수 없습니다. 서울의 현재 날씨를 확인하시려면 기상청 웹사이트나 날씨 앱을 참고하시기 바랍니다. 도움이 필요하시면 다른 질문이 있으신가요?
응답2: LangChain은 다양한 자연어 처리 작업을 수행할 수 있는 파이썬 라이브러리입니다. 주로 언어 모델과 상호작용하거나, 다양한 데이터 소스와 연결하여 복잡한 애플리케이션을 만드는 데 사용됩니다. LangChain을 사용하기 위해서는 몇 가지 단계를 따라야 합니다.

### 1. 환경 설정

먼저, Python 환경이 필요합니다. Python 3.7 이상이 설치되어 있어야 하며, `pip`를 사용하여 LangChain을 설치할 수 있습니다.

```bash
pip install langchain
```

### 2. 기본 사용법

LangChain을 사용하는 기본적인 예제를 보여드리겠습니다. 아래 코드는 LangChain을 사용하여 간단한 언어 모델을 호출하는 예입니다.

```python
from langchain import LLMChain
from langchain.llms import OpenAI

# OpenAI API 키 설정
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

# 언어 모델 인스턴스 생성
llm = OpenAI(model="text-davinci-003")

# LLMChain 생성
chain = LLMChain(llm=llm)

# 입력 텍스트
prompt = "Python의 장점은 무엇인가요?"

# 결과 생성
result = chain.run(prompt)

print(result)
```

### 3. 구성 요소

- **LLM (Language Model)**: OpenAI와 같은 특정 언어 모델을 사용하여 텍스트를 생성합니다.
- **LLMChain**: 입력 프롬프트를 받아 언어 모델을 호출하고, 결과를 반환합니다.

### 4. 다양한 기능

LangChain은 단순한 언어 

# LangChain의 LLM 응답 캐싱( In-Memory Caching, SQLiteCache )
- 동일한 질문은 저장해 뒀다가 응답에 재사용한다.

## In-Memory Caching

In [6]:
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# In-Memory Cache 설정
set_llm_cache(InMemoryCache())

In [9]:
# 캐시 사용 전 후 비교
# 같은 질문을 두 번 호출
query = "재미있는 유머 하나 알려줘"
# 첫 번째 호출: 캐시에 응답이 없으면 API 호출 발생
response1 = llm.invoke(query)
print("응답1:", response1.content)
# 두 번째 호출: 캐시에 응답이 있으면 API 호출 없이 캐시된 응답 사용
response2 = llm.invoke(query)
print("응답2:", response2.content)

응답1: 물론이죠! 다음은 간단한 유머입니다:

왜 컴퓨터는 바다에 빠지지 않을까요?

왜냐하면 "다이브(Dive)" 버튼이 없거든요! 

재미있으셨나요? 더 필요하시면 말씀해 주세요!
응답2: 물론이죠! 다음은 간단한 유머입니다:

왜 컴퓨터는 바다에 빠지지 않을까요?

왜냐하면 "다이브(Dive)" 버튼이 없거든요! 

재미있으셨나요? 더 필요하시면 말씀해 주세요!


In [12]:
# 실행시간 측정
import time

query = "오늘 점심 메뉴 추천해줘"
# 첫 번째 호출 시간
start = time.time()
llm.invoke(query)
end = time.time()
print(f"첫 번째 호출 시간: {end - start:.4f}초")
# 두 번째 호출 시간 
start = time.time()
llm.invoke(query)
end = time.time()
print(f"두 번째 호출 시간: {end - start:.4f}초")

첫 번째 호출 시간: 6.4144초
두 번째 호출 시간: 0.0007초


# SQLite 캐시( 디스크 기반 캐시 )

In [8]:
import os, time
from langchain_community.cache import SQLiteCache

# SQLite Cache 설정( 지정한 경로의 DB 파일을 생성 / 사용 )
set_llm_cache(SQLiteCache(database_path=".langchain.db")) # langchain.db 파일 생성

# 동일한 query를 두 번 호출해서 결과와 시간을 비교
query = "넌센스 퀴즈 하나와 정답을 알려줘."
# 첫 번째 호출: 캐시에 응답이 없으면 API 호출 발생
start = time.time()
response1 = llm.invoke(query)
end = time.time()
print("응답1:", response1.content)
print(f"첫 번째 호출 시간: {end - start:.4f}초")

print("-" * 100)

# 두 번째 호출: 캐시에 응답이 있으면 API 호출 없이 캐시된 응답 사용
start = time.time()
response2 = llm.invoke(query)
end = time.time()
print("응답2:", response2.content)
print(f"두 번째 호출 시간: {end - start:.4f}초")

응답1: 넌센스 퀴즈 하나 드릴게요!

**문제:** 
어떤 행성이 가장 많은 친구를 가지고 있을까요?

**정답:** 
화성! (화성은 '화'가 많아서 친구가 많다는 말장난이에요.) 

재미있죠? 또 궁금한 게 있으면 말씀해 주세요!
첫 번째 호출 시간: 0.0033초
----------------------------------------------------------------------------------------------------
응답2: 넌센스 퀴즈 하나 드릴게요!

**문제:** 
어떤 행성이 가장 많은 친구를 가지고 있을까요?

**정답:** 
화성! (화성은 '화'가 많아서 친구가 많다는 말장난이에요.) 

재미있죠? 또 궁금한 게 있으면 말씀해 주세요!
두 번째 호출 시간: 0.0011초
